<a href="https://colab.research.google.com/github/NeelkanthSingh/Auction_Website/blob/main/swiggy_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import re
import datetime

#Input the data

In [2]:
dis_str = pd.read_csv('/content/drive/MyDrive/Icecream Labs/Swiggy/discount_structure2.csv')  #Discount structure

In [3]:
swi_sta = pd.read_excel('/content/drive/MyDrive/Icecream Labs/Swiggy/December  20 Swiggy statement.xlsx', header=None) #Swiggy statement

In [4]:
ris_api = pd.read_csv('/content/drive/MyDrive/Icecream Labs/Swiggy/rista_api (11).csv') #Rista Api

#Preprocess the Synthetic data

##Discount structure

In [5]:
dis_str.head(6)

,Aggregator,Brands,Discount Coupon,Active (Y/N),Min Ord value1,Discount percentage1,Max Discount Amount1,Day1,Dates,MinTime1,MaxTime1
0,Swiggy,"Growfit,Ghar Ka Tadka",SwiggyIt 30,y,149,30,75,NaN,1-31,NaN,NaN
1,Swiggy,"Growfit,Ghar Ka Tadka",Jumbo,y,400,20,100,NaN,1-31,NaN,NaN
2,Swiggy,"Growfit,Ghar Ka Tadka",Party,y,800,15,300,"Mon,Tue,Thu,Fri",1-31,12:00 AM,4:00 PM
3,Swiggy,Growfit,Deal 60,y,0,15,120,"Wed,Fri,Sat,Sun","12,19,26,31",4:00 PM,11:59 PM
4,Swiggy,Ghar Ka Tadka,Deal 60,y,149,60,120,NaN,"13,20,27,31",NaN,NaN
5,Zomato,"Ghar Ka Tadka,Gunpowder Idlis",Deal 60,y,149,60,120,NaN,29,NaN,NaN


In [6]:
dis_str = dis_str[dis_str['Aggregator'] == "Swiggy"]  #Keep discount structure related to swiggy only

In [7]:
dis_str = dis_str.drop(labels=['Aggregator'],axis=1)  #Remove aggregator column, we know it only has swiggy value, It serves no purpose

In [8]:
dis_str['Discount Coupon']=['SWIGGYIT','JUMBO','PARTY','DEAL60','DEAL60'] #Capitalized discount coupon to keep them similar to those of Rista api

In [9]:
dis_str['Brands'] = pd.Series(['Grow,Ghar', 'Grow,Ghar', 'Grow,Ghar', 'Grow', 'Ghar', 'Ghar, Gunp', 'Gunt'])  #Changed them to keep them similar to those of Rista api

In [10]:
dis_str['Brands'] = dis_str['Brands'].apply(lambda x: list(map(str,x.split(','))))  #Make a list of brands, makes it easier to perform operation

In [11]:
def funct(dates):
  if re.search('-',dates):
    mini, maxi = map(int,dates.split('-'))
    return (mini, maxi)
  elif re.search(',',dates):
    dates = list(map(int,dates.split(',')))
  return dates

dis_str['Dates'] = dis_str['Dates'].apply(lambda x: funct(x))

#It converts dates from string format to list or tuple format which will later make it easier to check for dates 

In [12]:
dis_str['MaxTime1']=pd.to_datetime(dis_str['MaxTime1'], format='%I:%M %p')
dis_str['MinTime1']=pd.to_datetime(dis_str['MinTime1'], format='%I:%M %p')

# conveting from string to timestamp

In [13]:
dis_str['Day1'][dis_str['Day1'].notnull()] = dis_str['Day1'][dis_str['Day1'].notnull()].apply(lambda x: list(map(str,x.split(','))))

# converting days from str format to list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
def funct(brands,discount_coupon):
  result=list()
  for each in brands:
    result.append(discount_coupon+'-'+each)
  return result

# Made a temporary function, which is used to create another column with list of composite enteries of the type DISCOUNT_COUPON-BRAND

In [15]:
dis_str['Coupon Index'] = dis_str.apply(lambda x: funct(x['Brands'],x['Discount Coupon']),axis=1) #New column to store composite entry

##Swiggy Structure

In [16]:
swi_sta.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.98252e+06,79916,2.06244e+06,0,338238,1.7242e+06,81145.1,0,1.80535e+06,0,568127,0,220,568339,51150.2,51150.2,0,102300,0,670639,9272.75,0,6339,117,0,15728.8,1.11816e+06,8554.18,8554.18,0,12809.6,1.08824e+06,0,0,0,0,0,0,0,0,0,0
1,Week,kitchen id,Order Date,Order No,Order Status,Order Sub Type,Cancellation \nAttribution,Item's total\nA,Packing & Service charges\nB,Bill value\n(without taxes & discount)\nC=A+B,Merchant Trade Discount\nD,Merchant coupon Discount share\nE,Net bill value (without taxes)\nF = C - D - E,GST liability of \nMerchant G(i),CESS (collected from customer)\nG(ii),Customer payable\n(Net bill value after taxes ...,Cash prepayment at Merchant\nI,Swiggy platform service fee\nJ,Lead Generation fee @ 7.5%\nK,Call Center Service Fees \nL,Total Swiggy Service fee\n(without taxes)\nM =...,CGST\n(i)=O*9%,SGST\n(ii)=O*9%,IGST\n(iii)=O*18%,Total GST\nN(i) = (i) + (ii) + (iii),CESS \nN(ii),Total Swiggy service fee \n(including taxes)\n...,Buyer Cancellation Cost Sharing\nP = F*60%,Adhoc Adjustment\n Q,Refund To Customer\nR1,Delivery Fee(sponsored by merchant) \nR2,Comment\nR3,Total of other charges\nS = P - Q + R1 + R2,Net Payable Amount (before TCS and TDS deducti...,CTCS\nU1 = F*0.5%,STCS\nU2 = F*0.5%,ITCS\nU3 = F*01%,TDS\nU4,Net Payable Amount (after TCS and TDS deductio...,Order Type,Adhoc Remark,Original Payout Cycle,Remarks,MFR (Marked Food Ready) \napplicable on the or...,Cancellation Policy Applied,Coupon code applied,Is_replicated,Base order ID,Discount on Swiggy \nplatform service fee\nW
2,Week 1,228127,2020-12-01 11:17:30,89876850580,delivered,NaN,NaN,598,30,628,0,75,553,26.16,0,579.16,0,185.328,0,0,185.33,16.68,16.68,0,33.36,0,218.69,0,0,0,0,NaN,0,360.47,2.77,2.77,0,4.1475,350.78,regular,0,Current week,NaN,0,NaN,SWIGGYIT,N,NaN,0
3,Week 1,228127,2020-12-01 11:58:24,89879303982,delivered,NaN,NaN,149,10,159,0,0,159,7.46,0,166.46,0,53.264,0,0,53.26,4.79,4.79,0,9.58,0,62.84,0,0,0,0,NaN,0,103.6,0.8,0.8,0,1.1925,100.81,regular,0,Current week,NaN,0,NaN,NaN,N,NaN,0
4,Week 1,228127,2020-12-01 11:58:37,89879317155,delivered,NaN,NaN,358,17,375,0,74.99,300.01,14.16,0,314.17,0,100.531,0,0,100.53,9.05,9.05,0,18.1,0,118.63,0,0,0,0,NaN,0,195.53,1.5,1.5,0,2.25007,190.28,regular,0,Current week,NaN,0,NaN,SWIGGYIT,N,NaN,0


In [17]:
swi_sta = swi_sta[2:] #Remove the top two row 

In [18]:
swi_sta = swi_sta[[1,2,3,4,7,8,45]] #Keeping just the required column

In [19]:
swi_sta.set_axis(['Kitchen_id','order_date', 'order_no',"order_status",'gross_total','discount_value','coupon_code'], axis=1, inplace=True)

In [20]:
swi_sta.reset_index(drop=True, inplace=True) #Resetting the index

In [21]:
swi_sta['discount_value'][swi_sta['coupon_code'] == 'SWIGGYIT']=-75
swi_sta['discount_value'][swi_sta['coupon_code'] == 'DEAL60']=-120
swi_sta['discount_value'][swi_sta['coupon_code'] == 'JUMBO']=-100
swi_sta['discount_value'][swi_sta['coupon_code'] == 'PARTY']=-300

# Changing them to make some of them right cause few will  have max discount. Also to make them similar to Rista Api

In [22]:
def funct(dis_coupon):
  if(dis_coupon!='PARTY' and dis_coupon!='JUMBO' and dis_coupon!='DEAL60' and dis_coupon!='SWIGGYIT'):
    return True
  return False

# Temporary function to find out any coupon code other than those already present in Discount Structure

In [23]:
swi_sta['coupon_code'][swi_sta['coupon_code'].apply(lambda x: funct(x))]=np.NaN # Replacing those coupon codes with NaN

In [24]:
swi_sta['discount_value'][swi_sta['coupon_code'].apply(lambda x: funct(x))]=0 # Replacing discount value with 0

In [25]:
swi_sta['order_date'] = pd.to_datetime(swi_sta['order_date']) # changing to time format

In [26]:
swi_sta=swi_sta[:2000]  #Taking top 2000 for training

##Rista API

In [27]:
ris_api.head()

,branch_name,branch_code,brand_name,business_date,invoice_number,invoice_date,sale_status,order_source,source_order_number,gross_amount,discount,charge_amount,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,Korm,KM,Grow,01-03-2021,KM-47846,01-03-2021 10:26,Closed,NaN,0,250,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Korm,KM,Grow,01-03-2021,KM-47847,01-03-2021 10:27,Closed,NaN,0,250,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Korm,KM,Grow,01-03-2021,KM-47848,01-03-2021 10:28,Closed,NaN,0,285,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Korm,KM,Grow,01-03-2021,KM-47849,01-03-2021 10:29,Closed,NaN,0,250,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Korm,KM,Grow,01-03-2021,KM-47850,01-03-2021 10:29,Closed,NaN,0,305,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
for index,col in enumerate(ris_api.columns):
  if index>10:
    ris_api=ris_api.drop(col,axis=1)

  # Removing unnecessary column

In [29]:
ris_api.drop(labels=['branch_code','business_date'],axis=1,inplace=True)

In [30]:
count = dict()
def funct(source_od):
  if count.get(source_od) == None:
    count[source_od] = 1
  else:
    count[source_od] = count[source_od] + 1 

ris_api['source_order_number'].apply(lambda x: funct(x))
# check how many times a source order number is repeating cause we only need unique order numbers.

0       None
1       None
2       None
3       None
4       None
        ... 
5515    None
5516    None
5517    None
5518    None
5519    None
Name: source_order_number, Length: 5520, dtype: object

In [31]:
ris_api = ris_api[ris_api['source_order_number'].apply(lambda x: count[x]==1)] # Keep only those source order number which are unique

In [32]:
ris_api.reset_index(drop=True, inplace=True)

In [33]:
ris_api['invoice_date'] = pd.to_datetime(ris_api['invoice_date'])

In [34]:
ris_api=ris_api[:2000]  #Keeping just top 2000

##Making Swiggy statement and Rista Api similar

In [35]:
swi_sta['order_date'] = ris_api['invoice_date'].copy()
swi_sta['order_status'] = ris_api['sale_status'].copy()
swi_sta['order_no'] = ris_api['source_order_number']
swi_sta['gross_total'] = ris_api['gross_amount'].copy()
swi_sta['discount_value'] = ris_api['discount'].copy()

ris_api['Discount code'] = swi_sta['coupon_code'].copy()

# Making all these column enteries same because I consider they would be mostly same in real scenario

In [36]:
def funct(brand,discount_coupon):
  if type(discount_coupon) == str:
    return (discount_coupon+'-'+brand)
  else:
    return np.NaN

ris_api['Coupon Index'] = ris_api.apply(lambda x: funct(x['brand_name'],x['Discount code']),axis=1)

# Made a temporary function, which is used to create another column with composite enteries of the type DISCOUNT_COUPON-BRAND

#Show the data

In [37]:
dis_str.head()

,Brands,Discount Coupon,Active (Y/N),Min Ord value1,Discount percentage1,Max Discount Amount1,Day1,Dates,MinTime1,MaxTime1,Coupon Index
0,"[Grow, Ghar]",SWIGGYIT,y,149,30,75,NaN,"(1, 31)",NaT,NaT,"[SWIGGYIT-Grow, SWIGGYIT-Ghar]"
1,"[Grow, Ghar]",JUMBO,y,400,20,100,NaN,"(1, 31)",NaT,NaT,"[JUMBO-Grow, JUMBO-Ghar]"
2,"[Grow, Ghar]",PARTY,y,800,15,300,"[Mon, Tue, Thu, Fri]","(1, 31)",1900-01-01 00:00:00,1900-01-01 16:00:00,"[PARTY-Grow, PARTY-Ghar]"
3,[Grow],DEAL60,y,0,15,120,"[Wed, Fri, Sat, Sun]","[12, 19, 26, 31]",1900-01-01 16:00:00,1900-01-01 23:59:00,[DEAL60-Grow]
4,[Ghar],DEAL60,y,149,60,120,NaN,"[13, 20, 27, 31]",NaT,NaT,[DEAL60-Ghar]


In [38]:
swi_sta.head()

,Kitchen_id,order_date,order_no,order_status,gross_total,discount_value,coupon_code
0,228127,2021-01-03 11:08:00,97652300686,Closed,329,-75.00,SWIGGYIT
1,228127,2021-01-03 10:54:00,97651497829,Cancel,498,0.00,NaN
2,228127,2021-01-03 11:07:00,1725989242,Closed,249,0.00,SWIGGYIT
3,228127,2021-01-03 11:10:00,97652381277,Closed,344,-74.99,NaN
4,228127,2021-01-03 11:04:00,97652085234,Closed,209,0.00,NaN


In [39]:
ris_api.head()

,branch_name,brand_name,invoice_number,invoice_date,sale_status,order_source,source_order_number,gross_amount,discount,Discount code,Coupon Index
0,Doml,Grow,DMLRGF-60674,2021-01-03 11:08:00,Closed,swiggy,97652300686,329,-75.00,SWIGGYIT,SWIGGYIT-Grow
1,Korm,Grow,KM-47856,2021-01-03 10:54:00,Cancel,swiggy,97651497829,498,0.00,NaN,NaN
2,Korm,Ghar,KM-47857,2021-01-03 11:07:00,Closed,zomato,1725989242,249,0.00,SWIGGYIT,SWIGGYIT-Ghar
3,JP N,Grow,JPN-45067,2021-01-03 11:10:00,Closed,swiggy,97652381277,344,-74.99,NaN,NaN
4,Madh,Grow,MDH-30163,2021-01-03 11:04:00,Closed,swiggy,97652085234,209,0.00,NaN,NaN


#Validate the order

In [40]:
api_opr = ris_api[ris_api['order_source'] == 'swiggy']  #working on rista api for Swiggy order source

In [41]:
api_opr=api_opr[api_opr['sale_status'] == "Closed"] #Operating on rista api for Swiggy order source and completed order

In [42]:
api_opr.reset_index(drop=True, inplace=True)  #resetting the index

#Validate the discount

In [43]:
order_with_no_coupon = api_opr[api_opr['Discount code'].apply(lambda x: type(x) != str) ] #Picking those order who do not have coupon
order_with_no_coupon['discount'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
order_with_no_coupon.shape

(729, 11)

In [45]:
api_dis_merge = pd.merge(api_opr, dis_str,left_on='Discount code', right_on='Discount Coupon',  how='inner').drop('Discount Coupon',axis=1)

#Joined the two dataframe using coupon codes but brand may still be different. Also removed those order who do not have coupon code.

In [46]:
def funct(Index_x,Index_y):
  if type(Index_x) == str:
    if Index_x in Index_y:
      return True
    else:
      return False
  return False

api_dis_merge=api_dis_merge[api_dis_merge.apply(lambda x: funct(x['Coupon Index_x'],x['Coupon Index_y']),axis=1)]

#Here we remove those orders which though have the same coupon code but different brand name.

In [47]:
def coupon_active(active):
  if active.lower()=='y':
    return True
  return False

api_dis_merge=api_dis_merge[api_dis_merge['Active (Y/N)'].apply(lambda x: coupon_active(x))][api_dis_merge['gross_amount'] >= api_dis_merge['Min Ord value1']]

# Remove those order who do not have active coupon or gross total is less than the min value for which discount can be applied 

In [48]:
def funct(dates,invoice_date):
  if type(dates) == tuple:
    if invoice_date>=dates[0] and invoice_date<=dates[1]:
      return True
    else:
      return False
  elif type(dates)==list:
    for each in dates:
      if each == invoice_date:
        return True
    return False
  return True

api_dis_merge = api_dis_merge[api_dis_merge.apply(lambda x: funct(x['Dates'], x['invoice_date'].day),axis=1)]

# Remove those whose invoice days do not much with the coupon days

In [49]:
def funct(days,day):
  if type(days) != list:
    return True
  for i in days:
    if re.search(i.lower(),day.lower()):
      return True
  return False

api_dis_merge = api_dis_merge[api_dis_merge.apply(lambda x: funct(x['Day1'],x['invoice_date'].day_name()),axis=1)]

# Check if the day name marches for the coupon, if yes keep them otherwise discard them

In [50]:
def funct(mintime,maxtime,invoice_date):
  if type(mintime) != pd._libs.tslibs.timestamps.Timestamp:
    return True

  if type(maxtime) != pd._libs.tslibs.timestamps.Timestamp:
    return True

  if invoice_date.hour > mintime.hour and invoice_date.hour < maxtime.hour:
    return True
  elif invoice_date.hour == mintime.hour and invoice_date.hour < maxtime.hour:
    if invoice_date.minute >=mintime.minute:
      return True
    else:
      return False
  elif invoice_date.hour > mintime.hour and invoice_date.hour == maxtime.hour:
    if invoice_date.minute <= maxtime.minute:
      return True
    else:
      return False
  else:
    if invoice_date.minute <= maxtime.minute and invoice_date.minute >=mintime.minute:
      return True
    else:
      return False
  return False

# this function checks if the data is not null then whether the hour and minute satisfy.

api_dis_merge = api_dis_merge[api_dis_merge.apply(lambda x: funct(x['MinTime1'],x['MaxTime1'], x['invoice_date']),axis=1)]

# Check if the timing matches for the coupon and order

In [51]:
api_dis_merge = api_dis_merge[api_dis_merge.apply(lambda x: (abs(x['discount']) == x['Max Discount Amount1']) or ((abs(x['discount'])) == round((x['gross_amount'] * x['Discount percentage1'])/100, 1)), axis=1)]

In [52]:
api_dis_merge.shape

(320, 21)

#Final Result

In [53]:
for index,col in enumerate(api_dis_merge.columns):
  if index>9:
    api_dis_merge=api_dis_merge.drop(col,axis=1)

order_with_no_coupon.drop('Coupon Index', axis=1, inplace=True)

# Removing unnecessary column

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
Final_merge = api_dis_merge.append(order_with_no_coupon) # Appending orders those have corrrect discount or no discount at all

In [55]:
Final_merge.shape

(1049, 10)

In [56]:
last_check = pd.merge(Final_merge, swi_sta,left_on='source_order_number', right_on='order_no',  how='inner')  # Joining correct orders with swiggy statement for Reconciliation process

In [57]:
last_check.shape

(1049, 17)

In [58]:
last_check = last_check[last_check.apply(lambda x: (x['gross_amount']==x['gross_total']),axis=1)] # Checking if the gross value is same

In [59]:
last_check = last_check[last_check.apply(lambda x: (x['discount']==x['discount_value']),axis=1)]  # Checking if the discount is same

In [60]:
last_check.shape

(525, 17)